### Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import censusdata

### Get the variables that represents median income by race

Search for variables on median

In [2]:
search_criteria = "median family income"
income_variables = censusdata.search('acs5', 2018,'concept', search_criteria)
income_variables[0:2]

[('B10010_001E',
  'MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN',
  'Estimate!!Median family income in the past 12 months--!!Total'),
 ('B10010_002E',
  'MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN',
  'Estimate!!Median family income in the past 12 months--!!Grandparent householder and/or spouse responsible for grandchildren under 18 years (dollars)')]

Converts the list of tuples to a pandas data frame

In [3]:
dfIV = pd.DataFrame(income_variables, columns=["Variable","Concepts","Labels"])
dfIV.head()

,Variable,Concepts,Labels
0,B10010_001E,MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPA...,Estimate!!Median family income in the past 12 ...
1,B10010_002E,MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPA...,Estimate!!Median family income in the past 12 ...
2,B10010_003E,MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPA...,Estimate!!Median family income in the past 12 ...
3,B19113A_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN...,Estimate!!Median family income in the past 12 ...
4,B19113B_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN...,Estimate!!Median family income in the past 12 ...


Subset the data frame to exclude the row in the ***Variable*** field that is equal to "GEO_ID". Then displays using some style properties.

In [4]:
dfIV = dfIV[dfIV["Variable"] != "GEO_ID"]
dfIV.style.set_properties(subset=['Concepts'], **{'width-min': '300px'})

,Variable,Concepts,Labels
0,B10010_001E,MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN,Estimate!!Median family income in the past 12 months--!!Total
1,B10010_002E,MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN,Estimate!!Median family income in the past 12 months--!!Grandparent householder and/or spouse responsible for grandchildren under 18 years (dollars)
2,B10010_003E,MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN,Estimate!!Median family income in the past 12 months--!!Grandparent householder and/or spouse responsible for grandchildren under 18 years and no parent present (dollars)
3,B19113A_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) (WHITE ALONE HOUSEHOLDER),Estimate!!Median family income in the past 12 months (in 2018 inflation-adjusted dollars)
4,B19113B_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER),Estimate!!Median family income in the past 12 months (in 2018 inflation-adjusted dollars)
5,B19113C_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) (AMERICAN INDIAN AND ALASKA NATIVE ALONE HOUSEHOLDER),Estimate!!Median family income in the past 12 months (in 2018 inflation-adjusted dollars)
6,B19113D_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) (ASIAN ALONE HOUSEHOLDER),Estimate!!Median family income in the past 12 months (in 2018 inflation-adjusted dollars)
7,B19113E_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE HOUSEHOLDER),Estimate!!Median family income in the past 12 months (in 2018 inflation-adjusted dollars)
8,B19113F_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) (SOME OTHER RACE ALONE HOUSEHOLDER),Estimate!!Median family income in the past 12 months (in 2018 inflation-adjusted dollars)
9,B19113G_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) (TWO OR MORE RACES),Estimate!!Median family income in the past 12 months (in 2018 inflation-adjusted dollars)


### Put the variables that represent variables you want to retrieve in a list

In [5]:
cns_vars = [
    "B19113_001E","B19113A_001E","B19113B_001E","B19113C_001E",
    "B19113D_001E","B19113E_001E","B19113F_001E","B19113G_001E",
    "B19113H_001E","B19113I_001E"
    ] 

### Define the names you want to use

In [6]:
new_names = {
    'index':'Geography',"B19113_001E":"ALL","B19113A_001E":"WHITE",
    "B19113B_001E":"BLACK","B19113C_001E":"INDIAN","B19113D_001E":"ASIAN",
    "B19113E_001E":"PACIFIC","B19113F_001E":"OTHER","B19113G_001E":"MIX",
    "B19113H_001E":"JUST WHITE","B19113I_001E":"HISPANIC"
}

### Define the geographies you want data on

In [7]:
geographies = [('us', '*')]

### Fetch the data from the census api using the 5 year estimates for 2018 

In [8]:
MEDINC_BY_US_BY_RACE = censusdata.download(
    'acs5', 2018, censusdata.censusgeo(geographies), 
    cns_vars)

### Reest the index for the MEDINC_BY_STATE_BY_RACE data frame

In [9]:
MEDINC_BY_US_BY_RACE = MEDINC_BY_US_BY_RACE.reset_index()

### Rename the columns with more meaningful names

In [10]:
MEDINC_BY_US_BY_RACE = MEDINC_BY_US_BY_RACE.rename(columns=new_names)
MEDINC_BY_US_BY_RACE.head()

,Geography,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC
0,"United States: Summary level: 010, us:1",73965,78986,49270,49451,96439,65618,48058,66304,83246,51800


### Create a function to get the location name

In [11]:
def getLocation(row):
    return row["Geography"].name

### Sets the location field based on thee getLocation() field. Sets the "Place" field to the "Location" field

In [12]:
MEDINC_BY_US_BY_RACE["Location"] = MEDINC_BY_US_BY_RACE.apply(lambda row:  getLocation(row),axis=1)
MEDINC_BY_US_BY_RACE["Place"] = MEDINC_BY_US_BY_RACE["Location"]

In [13]:
MEDINC_BY_US_BY_RACE.head()

,Geography,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC,Location,Place
0,"United States: Summary level: 010, us:1",73965,78986,49270,49451,96439,65618,48058,66304,83246,51800,United States,United States


### Sets the Query field

In [14]:
MEDINC_BY_US_BY_RACE["Query Type"] = "Country"

### Get the median income data at the ***state*** level

In [15]:
# Define the geographies you want data on
geographies = [('state', '*')]

# Fetch the data from the census api using the 5 year estimates for 
# 2018 
MEDINC_BY_STATE_BY_RACE = censusdata.download(
    'acs5', 2018, censusdata.censusgeo(geographies), 
    cns_vars)

# Reest the index for the MEDINC_BY_STATE_BY_RACE data frame
MEDINC_BY_STATE_BY_RACE = MEDINC_BY_STATE_BY_RACE.reset_index()

# Rename the columns with more meaningful names
MEDINC_BY_STATE_BY_RACE = MEDINC_BY_STATE_BY_RACE.rename(columns=new_names)

# Create a function to get the location name
def getLocation(row):
    return row["Geography"].name

# Sets the location field based on thee getLocation() field. Sets the "Place" field to the "Location" field
MEDINC_BY_STATE_BY_RACE["Location"] = MEDINC_BY_STATE_BY_RACE.apply(lambda row:  getLocation(row),axis=1)
MEDINC_BY_STATE_BY_RACE["Place"] = MEDINC_BY_STATE_BY_RACE["Location"]

# Sets the Query field
MEDINC_BY_STATE_BY_RACE["Query Type"] = "State"

In [16]:
MEDINC_BY_STATE_BY_RACE

,Geography,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC,Location,Place,Query Type
0,"Minnesota: Summary level: 040, state:27",86204,90511,41891,41988,82767,73036,44056,64187,91432,49426,Minnesota,Minnesota,State
1,"Mississippi: Summary level: 040, state:28",54933,67910,36083,40196,69004,50160,41863,48971,68466,44566,Mississippi,Mississippi,State
2,"Missouri: Summary level: 040, state:29",67612,70959,43905,55236,88419,53594,39608,53025,71390,50307,Missouri,Missouri,State
3,"Montana: Summary level: 040, state:30",68139,70097,47050,40959,90125,28469,62900,53877,70522,52583,Montana,Montana,State
4,"Nebraska: Summary level: 040, state:31",75123,77802,42277,43295,68654,52813,47671,51224,80477,48565,Nebraska,Nebraska,State
5,"Nevada: Summary level: 040, state:32",68700,74562,48754,49846,74479,66710,47477,65483,80394,52317,Nevada,Nevada,State
6,"New Hampshire: Summary level: 040, state:33",91245,91877,62139,52099,95765,55673,61875,78702,92543,59101,New Hampshire,New Hampshire,State
7,"New Jersey: Summary level: 040, state:34",98047,106497,63387,65906,127067,48563,48587,81274,114801,56910,New Jersey,New Jersey,State
8,"New Mexico: Summary level: 040, state:35",59175,63383,50871,37741,73728,61850,42407,59913,77590,47703,New Mexico,New Mexico,State
9,"New York: Summary level: 040, state:36",80419,92553,58406,50108,77938,60295,43104,63412,96047,50984,New York,New York,State


### Get the median income data at the ***place*** level

In [17]:
# Define a new set of geographies based on places
geographies = [('state', '*'),('place','*')]

# Fetch the data from the census api
MEDINC_BY_STATE_BY_PLACE_BY_RACE = censusdata.download(
    'acs5', 2018, censusdata.censusgeo(geographies), 
    cns_vars)

# Reset your index
MEDINC_BY_STATE_BY_PLACE_BY_RACE = MEDINC_BY_STATE_BY_PLACE_BY_RACE.reset_index()

# Give the columns more meaningful names
MEDINC_BY_STATE_BY_PLACE_BY_RACE = MEDINC_BY_STATE_BY_PLACE_BY_RACE.rename(columns=new_names)

# Sets the "Place" field using the getLocation() function
MEDINC_BY_STATE_BY_PLACE_BY_RACE["Place"] = MEDINC_BY_STATE_BY_PLACE_BY_RACE.apply(lambda row:  getLocation(row),axis=1)

# Create a function to get the State out of the "Place" field and uses it to set the "Location" field
def getState(place):
    return place.split(",",1)[1]
MEDINC_BY_STATE_BY_PLACE_BY_RACE["Location"] = MEDINC_BY_STATE_BY_PLACE_BY_RACE["Place"].apply(lambda place: getState(place))

# Sets the Query field
MEDINC_BY_STATE_BY_PLACE_BY_RACE["Query Type"] = "Place"

In [18]:
# Get the shape of the data frame
MEDINC_BY_STATE_BY_PLACE_BY_RACE.shape

(29573, 14)

In [19]:
MEDINC_BY_STATE_BY_PLACE_BY_RACE.head()

,Geography,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC,Place,Location,Query Type
0,"Abanda CDP, Alabama: Summary level: 160, state...",41346.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,"Abanda CDP, Alabama",Alabama,Place
1,"Abbeville city, Alabama: Summary level: 160, s...",53036.0,71563.0,35000.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,71563.0,-666666666.0,"Abbeville city, Alabama",Alabama,Place
2,"Adamsville city, Alabama: Summary level: 160, ...",52846.0,60236.0,51696.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,60236.0,-666666666.0,"Adamsville city, Alabama",Alabama,Place
3,"Addison town, Alabama: Summary level: 160, sta...",59167.0,59167.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,59167.0,-666666666.0,"Addison town, Alabama",Alabama,Place
4,"Akron town, Alabama: Summary level: 160, state...",36000.0,60375.0,24583.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,60375.0,-666666666.0,"Akron town, Alabama",Alabama,Place


### Combines the contents from the MEDINC_BY_STATE_BY_PLACE_BY_RACE data frame and MEDINC_BY_STATE_BY_RACE data frame into one data frame.

In [21]:
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE = MEDINC_BY_US_BY_RACE

MEDINC_BY_LOCATION_BY_PLACE_BY_RACE = \
    MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.append(MEDINC_BY_STATE_BY_RACE, ignore_index = True)

MEDINC_BY_LOCATION_BY_PLACE_BY_RACE = \
    MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.append(MEDINC_BY_STATE_BY_PLACE_BY_RACE, ignore_index = True)
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.head()

,Geography,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC,Location,Place,Query Type
0,"United States: Summary level: 010, us:1",73965.0,78986.0,49270.0,49451.0,96439.0,65618.0,48058.0,66304.0,83246.0,51800.0,United States,United States,Country
1,"Minnesota: Summary level: 040, state:27",86204.0,90511.0,41891.0,41988.0,82767.0,73036.0,44056.0,64187.0,91432.0,49426.0,Minnesota,Minnesota,State
2,"Mississippi: Summary level: 040, state:28",54933.0,67910.0,36083.0,40196.0,69004.0,50160.0,41863.0,48971.0,68466.0,44566.0,Mississippi,Mississippi,State
3,"Missouri: Summary level: 040, state:29",67612.0,70959.0,43905.0,55236.0,88419.0,53594.0,39608.0,53025.0,71390.0,50307.0,Missouri,Missouri,State
4,"Montana: Summary level: 040, state:30",68139.0,70097.0,47050.0,40959.0,90125.0,28469.0,62900.0,53877.0,70522.0,52583.0,Montana,Montana,State


### Replaces -666666666 with NaN

In [22]:
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE = MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.replace(-666666666, np.nan)
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.head(100)

,Geography,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC,Location,Place,Query Type
0,"United States: Summary level: 010, us:1",73965.0,78986.0,49270.0,49451.0,96439.0,65618.0,48058.0,66304.0,83246.0,51800.0,United States,United States,Country
1,"Minnesota: Summary level: 040, state:27",86204.0,90511.0,41891.0,41988.0,82767.0,73036.0,44056.0,64187.0,91432.0,49426.0,Minnesota,Minnesota,State
2,"Mississippi: Summary level: 040, state:28",54933.0,67910.0,36083.0,40196.0,69004.0,50160.0,41863.0,48971.0,68466.0,44566.0,Mississippi,Mississippi,State
3,"Missouri: Summary level: 040, state:29",67612.0,70959.0,43905.0,55236.0,88419.0,53594.0,39608.0,53025.0,71390.0,50307.0,Missouri,Missouri,State
4,"Montana: Summary level: 040, state:30",68139.0,70097.0,47050.0,40959.0,90125.0,28469.0,62900.0,53877.0,70522.0,52583.0,Montana,Montana,State
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Bellamy CDP, Alabama: Summary level: 160, stat...",29205.0,NaN,37793.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama,"Bellamy CDP, Alabama",Place
96,"Belle Fontaine CDP, Alabama: Summary level: 16...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alabama,"Belle Fontaine CDP, Alabama",Place
97,"Benton town, Alabama: Summary level: 160, stat...",64688.0,64688.0,NaN,NaN,NaN,NaN,NaN,NaN,64688.0,NaN,Alabama,"Benton town, Alabama",Place
98,"Berry town, Alabama: Summary level: 160, state...",56250.0,56806.0,NaN,NaN,NaN,NaN,NaN,NaN,56806.0,NaN,Alabama,"Berry town, Alabama",Place


### Choose the columns that you want to keep in the order you want them

In [23]:
colnames = ['Query Type', 'Location', 'Place', 'ALL', 'WHITE', 'BLACK', 'INDIAN', 'ASIAN', 'PACIFIC','OTHER', 'MIX', 'JUST WHITE', 'HISPANIC']
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE = MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.loc[:,colnames]
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.head()

,Query Type,Location,Place,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC
0,Country,United States,United States,73965.0,78986.0,49270.0,49451.0,96439.0,65618.0,48058.0,66304.0,83246.0,51800.0
1,State,Minnesota,Minnesota,86204.0,90511.0,41891.0,41988.0,82767.0,73036.0,44056.0,64187.0,91432.0,49426.0
2,State,Mississippi,Mississippi,54933.0,67910.0,36083.0,40196.0,69004.0,50160.0,41863.0,48971.0,68466.0,44566.0
3,State,Missouri,Missouri,67612.0,70959.0,43905.0,55236.0,88419.0,53594.0,39608.0,53025.0,71390.0,50307.0
4,State,Montana,Montana,68139.0,70097.0,47050.0,40959.0,90125.0,28469.0,62900.0,53877.0,70522.0,52583.0


### Creates a boolean series that returns true if all of the race fields are null and returns False otherwise. This series will be used to subset the MEDINC_BY_LOCATION_BY_PLACE_BY_RACE by removing all records where the "race" fields are all nulls.

In [24]:
logic = \
    ~(
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["ALL"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["WHITE"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["BLACK"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["INDIAN"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["ASIAN"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["PACIFIC"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["OTHER"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["MIX"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["JUST WHITE"] .isnull() & \
        MEDINC_BY_LOCATION_BY_PLACE_BY_RACE["HISPANIC"] .isnull()
    )

In [25]:
logic

0         True
1         True
2         True
3         True
4         True
         ...  
29621    False
29622    False
29623    False
29624    False
29625    False
Length: 29626, dtype: bool

In [ ]:
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.shape

In [26]:
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE = MEDINC_BY_LOCATION_BY_PLACE_BY_RACE[logic]

In [27]:
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.shape

(27287, 13)

In [28]:
MEDINC_BY_LOCATION_BY_PLACE_BY_RACE.head()

,Query Type,Location,Place,ALL,WHITE,BLACK,INDIAN,ASIAN,PACIFIC,OTHER,MIX,JUST WHITE,HISPANIC
0,Country,United States,United States,73965.0,78986.0,49270.0,49451.0,96439.0,65618.0,48058.0,66304.0,83246.0,51800.0
1,State,Minnesota,Minnesota,86204.0,90511.0,41891.0,41988.0,82767.0,73036.0,44056.0,64187.0,91432.0,49426.0
2,State,Mississippi,Mississippi,54933.0,67910.0,36083.0,40196.0,69004.0,50160.0,41863.0,48971.0,68466.0,44566.0
3,State,Missouri,Missouri,67612.0,70959.0,43905.0,55236.0,88419.0,53594.0,39608.0,53025.0,71390.0,50307.0
4,State,Montana,Montana,68139.0,70097.0,47050.0,40959.0,90125.0,28469.0,62900.0,53877.0,70522.0,52583.0


### Python resources

- Python for Informatics: http://www.py4inf.com/
- Link to Data School YouTube channel
- Link to the CensusData github site